## Load packages and data

In [1]:
# remove hashtag and run code below to install packages

#!pip install -U sentence-transformers
#!pip install --user annoy

In [7]:
import numpy as np
import pandas as pd
import json
import copy
import time
import sqlalchemy as db
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate playlist vectors using sentence transformers and load into annoy index


In [4]:
####use SQL to generate a list of pid's in training set
pid_train = []

In [5]:
#dimension of vectors the model uses
f = model.get_sentence_embedding_dimension()

#build database we will load vectors in
#angular means we are using cosine similarty metric
t = AnnoyIndex(f , 'angular')

In [6]:
for pid in pid_train:
    #use SQL to pull name of playlist of given pid
    #store in the variable name pid_name
    pid_name 
    
    #construct vector from playlist name
    v = model.encode(pid_name.lower().strip())
    
    #add playlist to annot
    t.add_item(pid, v)

In [ ]:
#number of trees for search
t.build(1024)
t.save('playlist_vectors.ann')